# 模型增量更新与每日预测引擎

---

### **目标**
本 Notebook 是项目的**每日运行脚本**，用于实现模型的在线学习（热更新）和生成新的交易信号。

### **工作流程**
1.  **环境设置与目标选定**: 导入库，加载配置，并指定要进行增量更新的股票。
2.  **加载所有模型**: 加载基础模型 (LGBM, LSTM) 和**上一次训练好**的融合模型 (Fuser)。
3.  **模拟获取新知**: 
    - **获取昨日预测**: 模拟从数据库或日志中读取昨天生成的 L1 模型预测值。
    - **获取今日真实标签**: 获取今天的市场数据，并计算出昨日预测所对应的真实结果 `y_true`。
4.  **执行增量训练**: 使用这对 `(昨日预测, 今日真实标签)` 对融合模型进行 `partial_train`，使其权重得到微调，并自动保存更新后的模型。
5.  **生成今日新预测 (可选)**: 
    - 获取截至**今日收盘**的最新特征数据。
    - 使用基础模型生成今日的 L1 预测。
    - 使用**刚刚更新过**的融合模型，生成用于**明天**交易的最终决策建议。

## 1. 环境设置与目标选定

In [ ]:
import sys, yaml, pandas as pd, joblib, torch
from pathlib import Path

# --- 模块导入 ---
try:
    from data_process.get_data import initialize_apis, shutdown_apis, get_full_feature_df
    from model_builders.model_fuser import ModelFuser
    from model_builders.lstm_builder import LSTMModel
    print("INFO: Project modules imported successfully.")
except ImportError as e:
    print(f"WARNNING: Import failed: {e}. Adding project root...")
    project_root = str(Path().resolve()); sys.path.append(project_root) if project_root not in sys.path else None
    from data_process.get_data import initialize_apis, shutdown_apis, get_full_feature_df
    from model_builders.model_fuser import ModelFuser
    from model_builders.lstm_builder import LSTMModel
    print("INFO: Re-imported successfully.")

# --- 加载配置文件 ---
CONFIG_PATH = 'configs/config.yaml'
try:
    with open(CONFIG_PATH, 'r', encoding='utf-8') as f: config = yaml.safe_load(f)
    print(f"SUCCESS: Config loaded from '{CONFIG_PATH}'.")
except FileNotFoundError:
    print(f"ERROR: Config file not found."); config = {}

# --- 设定要进行增量更新的股票 --- 
TARGET_TICKER = '600519.SH'
stock_info = next((s for s in config.get('stocks_to_process', []) if s['ticker'] == TARGET_TICKER), None)
if stock_info:
    TARGET_KEYWORD = stock_info.get('keyword', TARGET_TICKER)
    print(f"--- 目标股票已设定: {TARGET_KEYWORD} ({TARGET_TICKER}) ---")
else:
    print(f"ERROR: 在配置文件中未找到股票 {TARGET_TICKER} 的信息！")

## 2. 加载所有模型 (L1 基础模型 + L2 融合模型)

In [ ]:
base_models = {}
base_scalers = {}
fuser = None
all_models_loaded = False

if stock_info:
    model_dir = Path(config.get('global_settings', {}).get('model_dir', 'models')) / TARGET_TICKER
    models_to_load = config.get('global_settings', {}).get('models_to_train', ['lgbm', 'lstm'])
    
    # --- 1. 加载基础模型 ---
    for model_type in models_to_load:
        model_files = sorted(model_dir.glob(f"{model_type}_model_*.p*t"))
        if not model_files: print(f"ERROR: 未找到 {model_type.upper()} 模型文件。"); break
        
        latest_model_file = model_files[-1]
        version_timestamp = latest_model_file.stem.split('_')[-1]
        latest_scaler_file = model_dir / f"{model_type}_scaler_{version_timestamp}.pkl"

        try:
            if model_type == 'lgbm':
                base_models[model_type] = joblib.load(latest_model_file)
            elif model_type == 'lstm':
                lstm_cfg = {**config.get('default_model_params',{}), **stock_info}.get('lstm_params',{})
                # 这里的 input_size 暂时用一个通用值，稍后会从数据中动态获取
                model_instance = LSTMModel(input_size=23, hidden_size_1=lstm_cfg.get('units_1',64), hidden_size_2=lstm_cfg.get('units_2',32), dropout=lstm_cfg.get('dropout',0.2))
                model_instance.load_state_dict(torch.load(latest_model_file))
                model_instance.eval()
                base_models[model_type] = model_instance
            
            base_scalers[model_type] = joblib.load(latest_scaler_file)
            print(f"SUCCESS: 成功加载 {model_type.upper()} 版本 '{version_timestamp}' 的模型和 Scaler。")
        except Exception as e:
            print(f"ERROR: 加载 {model_type.upper()} 构件失败: {e}"); break
    
    # --- 2. 加载融合模型 ---
    if len(base_models) == len(models_to_load):
        fuser = ModelFuser(TARGET_TICKER, config)
        if fuser.load(): # load() 方法会自己打印日志
            all_models_loaded = True
        else:
            print("ERROR: 融合模型加载失败，无法进行增量更新。请先运行主训练流程中的融合模型训练。")

if all_models_loaded:
    print("\n--- 所有模型构件已成功加载！---")

## 3. 模拟获取“新知”：昨日预测与今日真实标签

In [ ]:
new_data_for_update = None

if all_models_loaded:
    # 在真实系统中，这部分会从数据库、API或日志文件中获取
    # 这里我们进行模拟：
    # 1. 获取包含最近两天数据的特征集
    # 2. 用倒数第二行数据，模拟生成“昨日的预测”
    # 3. 从最后一行数据，计算出“今日的真实标签”
    print("--- 正在模拟获取新的训练样本... ---")
    try:
        initialize_apis(config)
        # 获取近期数据
        recent_df = get_full_feature_df(TARGET_TICKER, config, keyword=TARGET_KEYWORD, prediction_mode=True)
        if recent_df is not None and len(recent_df) >= 2:
            yesterday_features = recent_df.iloc[-2:-1] # 倒数第二行
            today_data = recent_df.iloc[-1:]       # 最后一行
            
            # --- 模拟生成昨日预测 --- 
            label_col = config.get('global_settings', {}).get('label_column', 'label_return')
            feature_cols = [c for c in yesterday_features.columns if c != label_col]
            X_yesterday = yesterday_features[feature_cols]
            
            # LGBM 预测
            X_scaled_lgbm = base_scalers['lgbm'].transform(X_yesterday)
            pred_lgbm = base_models['lgbm']['q_0.5'].predict(X_scaled_lgbm)[0]

            # LSTM 预测
            lstm_cfg = {**config.get('default_model_params',{}), **stock_info}.get('lstm_params',{})
            seq_len = lstm_cfg.get('sequence_length', 60)
            pseudo_sequence = pd.concat([X_yesterday] * seq_len, ignore_index=True)
            X_scaled_lstm = base_scalers['lstm'].transform(pseudo_sequence)
            X_tensor_lstm = torch.from_numpy(X_scaled_lstm).unsqueeze(0).float()
            with torch.no_grad(): pred_lstm = base_models['lstm'](X_tensor_lstm).item()
            
            print(f"模拟的昨日预测 -> pred_lgbm: {pred_lgbm:.6f}, pred_lstm: {pred_lstm:.6f}")
            
            # --- 获取今日真实标签 --- 
            # 注意：在实盘中，labeling_horizon=30 的真实标签需要30天后才能知道。
            # 为了能每日更新，我们通常会使用一个“代理”标签，例如次日收益率。
            # 这里我们为了演示，直接使用数据中的 `label_return` 作为近似。
            y_true_today = today_data[label_col].iloc[0]
            print(f"获取的今日真实标签 (y_true): {y_true_today:.6f}")
            
            # --- 组合成新的训练样本 ---
            new_data_for_update = pd.DataFrame([{
                'pred_lgbm': pred_lgbm,
                'pred_lstm': pred_lstm,
                'y_true': y_true_today
            }])
    finally:
        shutdown_apis()


## 4. 执行增量训练

In [ ]:
if fuser is not None and new_data_for_update is not None:
    # 检查新数据是否有效
    if not new_data_for_update.isnull().values.any():
        # 调用增量训练方法
        fuser.partial_train(new_data_for_update)
    else:
        print("WARNNING: 新的训练样本包含无效值，跳过本次增量更新。")
else:
    print("模型或新数据未准备好，跳过增量训练。")

## 5. (可选) 生成今日新预测

使用刚刚经过增量更新的 `fuser` 模型，来生成用于明天交易的最终信号。

In [ ]:
if fuser is not None and recent_df is not None and len(recent_df) > 0:
    print("--- 使用增量更新后的模型生成新预测 ---")
    
    # 1. 提取今日的最新特征
    today_features = recent_df.iloc[-1:]
    label_col = config.get('global_settings', {}).get('label_column', 'label_return')
    feature_cols = [c for c in today_features.columns if c != label_col]
    X_today = today_features[feature_cols]
    
    # 2. 生成 L1 预测
    X_scaled_lgbm = base_scalers['lgbm'].transform(X_today)
    pred_lgbm_today = base_models['lgbm']['q_0.5'].predict(X_scaled_lgbm)[0]
    
    lstm_cfg = {**config.get('default_model_params',{}), **stock_info}.get('lstm_params',{})
    seq_len = lstm_cfg.get('sequence_length', 60)
    pseudo_sequence = pd.concat([X_today] * seq_len, ignore_index=True)
    X_scaled_lstm = base_scalers['lstm'].transform(pseudo_sequence)
    X_tensor_lstm = torch.from_numpy(X_scaled_lstm).unsqueeze(0).float()
    with torch.no_grad(): pred_lstm_today = base_models['lstm'](X_tensor_lstm).item()
    
    # 3. 使用 fuser 进行最终融合
    final_signal = fuser.predict({
        'pred_lgbm': pred_lgbm_today,
        'pred_lstm': pred_lstm_today
    })
    
    # 4. 生成建议报告
    # (这里复用了 Prophet.ipynb 中的报告生成逻辑)
    direction = '看涨 (BUY)' if final_signal > 0 else '看跌 (SELL)'
    print(f"\n--- 为 {TARGET_KEYWORD} 生成的明日交易建议 ---")
    print(f"核心观点: {direction}")
    print(f"信号强度 (预期收益): {final_signal:.4%}")
    print("="*80)
    
else:
    print("\n未能生成新预测。")